# Training a Deep Network in PyTorch

[link](https://ut.philkr.net/deeplearning/deep_networks/training_a_deep_network_in_pytorch/)

In [1]:
import torchvision
import torch
from PIL import Image

device = torch.device("mps")

In [2]:
size = (128,128)
transform = torchvision.transforms.Compose([torchvision.transforms.Resize(size),torchvision.transforms.ToTensor()])
train_dataset = torchvision.datasets.Flowers102(root='./flowers', split = 'train', download=True, transform=transform)
test_dataset = torchvision.datasets.Flowers102(root='./flowers', split = 'test', download=True, transform=transform)
train_dataset


Dataset Flowers102
    Number of datapoints: 1020
    Root location: ./flowers
    split=train
    StandardTransform
Transform: Compose(
               Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

## Data Loader

In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=len(test_dataset), shuffle=False)

## Model

In [4]:
from mlp import MLP
model = MLP(input_dim=size[0]*size[1]*3,
            hidden_dim=[512,512,512],
            output_dim=102
        ).to(device)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
model.train()

MLP(
  (network): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=49152, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): ReLU()
    (7): Linear(in_features=512, out_features=102, bias=True)
  )
)

### Batch Gradient Descent

In [8]:
for epoch in range(100):
    losses= []
    for img,label in train_loader:
        img = img.to(device)
        label = label.to(device)
        pred_y = model(img)
        optimizer.zero_grad()
        loss_value = loss(pred_y, label)
        loss_value.backward()
        optimizer.step()
        losses.append(loss_value.item())
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, loss: {sum(losses)/len(losses)}")

Epoch 0, loss: 4.632189214229584
Epoch 10, loss: 3.8514105677604675
Epoch 20, loss: 3.2630146592855453
Epoch 30, loss: 2.768210843205452
Epoch 40, loss: 1.8873515129089355
Epoch 50, loss: 1.4114683978259563
Epoch 60, loss: 0.18852922320365906
Epoch 70, loss: 0.006817961693741381
Epoch 80, loss: 0.0039718886982882395
Epoch 90, loss: 0.0028544429369503632


## Evaluation

It is overfitting

In [ ]:
for test_images, test_labels in test_loader:
    test_images, test_labels = test_images.to(device), test_labels.to(device)
    prred_test = model(test_images)
    accuracy = ((prred_test.argmax(dim=1) == test_labels).float().mean()).item()
print(f"Test accuracy: {accuracy}")

Test accuracy: 0.01967799663543701
